In [ ]:
import time
from IPython.display import clear_output, display

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import random
%matplotlib inline

In [ ]:
# Utils

# Fonction pour générer la droite représentant notre modèle
def getHypothesisForPLot(theta):
    return pd.DataFrame({'x':np.arange(0, 12000, 100)/1000,
                         'y':[hypothesis(x,theta)/1000 for x in np.arange(0, 12000, 100)]})

# on plot les données avec l'hypothèse correpondant à la valeur de theta 
#    ainsi que l'évolution dans la courbe de J(theta) en fonction de theta
# On rajoute également la valeur de J(theta) en fonction du temps qui va nous servir à 
#   débuger notre algorithme

def plotData(ax,x,y,theta,yhat,gradDescentEvol):
    ax.plot(x,y,'o',label='data')
    ax.plot(getHypothesisForPLot(theta).x,getHypothesisForPLot(theta).y,'r',label='hypothèse')
    for i in range(x.shape[0]):
        ax.plot((x[i],x[i]), (min(y[i],yhat[i]),max(y[i],yhat[i])), 'k-')
    ax.legend(fontsize=12)

def plotCostFunction(ax,x,y,theta,gradDescentEvol,thetaInit):
#    thetaRange = np.arange(abs(thetaInit)-100,abs(thetaInit)+100,0.1)
    thetaRange = np.arange(80-100,80+100,0.1)
    costFctEvol = pd.DataFrame({'theta':thetaRange,
                                'cost':[costFunction(y,hypothesis(x,genTheta))
                                        for genTheta in thetaRange]})

    ax.plot(costFctEvol.theta,costFctEvol.cost,label='J(theta)')
    for i in range(gradDescentEvol.shape[0]):
        ax.plot(gradDescentEvol.theta[i],gradDescentEvol.J[i],'ro')
    for i in range(gradDescentEvol.shape[0]-1):
        ax.plot((gradDescentEvol.theta[i],gradDescentEvol.theta[i+1]),
                (gradDescentEvol.J[i],gradDescentEvol.J[i+1]),'k-',lw=1)
    ax.legend(fontsize=12)

def plotCostFunctionEvol(ax,gradDescentEvol):
    ax.plot(np.arange(gradDescentEvol.shape[0]),gradDescentEvol.J,label='J(theta)')
    ax.legend(fontsize=12)
    
def plotCostFunctionEvolwTest(ax,gradDescentEvol):
    ax.plot(np.arange(gradDescentEvol.shape[0]),gradDescentEvol.Jtrain,label='J(train)')
    ax.plot(np.arange(gradDescentEvol.shape[0]),gradDescentEvol.Jtest,label='J(test)')
    ax.legend(fontsize=15)

# 1) Get data

In [ ]:
# Get data
data = pd.read_csv('data/graphicCards.csv')
data = data[['memory (Go)','price (euros)']].dropna()
data.columns = ['x1','y']
data.x1 = data.x1
print(data.shape)
data.head()

In [ ]:
#Plot data
fig, ax = plt.subplots(figsize=(16,10))
plt.plot(data.x1,data.y,'o', ms=2)
plt.xlabel('GPU (Go)',fontsize=15)
plt.ylabel('prix (€)',fontsize=15)
plt.show();

# 2) Contruire un modéle pour nos données


* ### Soit: $x_{1}$ la valeur de GPU de nos $m$ carte graphiques, et $y$ leur prix

* ### On cherche à déterminer le modèle pour prédire un prix $\hat{y}$ à partir $x_{1}$: 

## $\hat{y} = h_{\theta}(x_{1})$


* ### On défini le paramètre $\theta_{1}$ qui va lier $x_{1}$ à $\hat{y}$: 

## $ h_{\theta}(x) = \theta_{1} x_{1}$


* ### Rappel math: fonction linéaire $f(x) = kx$

In [ ]:
# Définir notre hypothèse (fonction)

def hypothesis(x,theta):
    return np.dot(x,theta)

In [ ]:
# On génère aléatoirement une valeur de départ pour le paramètre theta1 de notre modèle

theta = np.random.rand()
print(theta)

In [ ]:
# On plot les données avec notre hypothèse ...

fig, ax = plt.subplots(figsize=(16,10))
plt.plot(data.x1, data.y,'o',label='data',ms=2)
plt.plot(getHypothesisForPLot(theta).x,getHypothesisForPLot(theta).y ,'r',label='hypothèse')
plt.xlabel('GPU (Go)',fontsize=15)
plt.ylabel('prix (€)',fontsize=15)
plt.title("C'est pas ça ....",fontsize=15)
plt.legend(fontsize=15)
plt.show();

print("theta = %f" % theta)

# 3) Tester la pertinence de notre modèle: la fonction de coût

* ### $J(\theta)$: Véracité de notre modèle

* ### Somme Quadratique des erreurs: 

## $J(\theta) = \frac{1}{2m} \displaystyle\sum_{i=0}^{m}(\hat{y}^{(i)} - y^{(i)})^{2}$

In [ ]:
# On définit notre fonction de coût: somme quadratique (eg: on somme les carré)

def costFunction(y,yhat):
    return np.square(yhat - y).sum()/(2*y.shape[0])

In [ ]:
# Prix prédis par notre modèle (avec un theta choisi pour illustrer) pour chaque exemple

theta = 80
costFctData = data.iloc[np.random.randint(0,data.shape[0],10)]
costFctData.index = np.arange(10)
yhat = hypothesis(costFctData.x1,theta)

In [ ]:
#Comment fonctionne la fonction de coût: on somme le carré de toute les barre noire

fit, ax = plt.subplots(figsize=(16,10))
plt.plot(costFctData.x1,costFctData.y,'o',label='data')
plt.plot(getHypothesisForPLot(theta).x,getHypothesisForPLot(theta).y,'r',label='hypothèse')
for i in range(costFctData.shape[0]):
    plt.plot((costFctData.x1[i],costFctData.x1[i]), (min(costFctData.y[i],yhat[i]),max(costFctData.y[i],yhat[i])), 'k-')
plt.xlabel('GPU (Go)',fontsize=15)
plt.ylabel('prix (€)',fontsize=15)
plt.legend(fontsize=15)
plt.show();

print("theta = %f" % theta)
print("J(theta) = %f" % costFunction(costFctData.y,yhat))

# 4) À quoi ressemble J(theta) en fonction de theta1

In [ ]:
# Calculons (brutalement) la valeur de J(theta) dans un intervale de valeur de theta1 
#     pour observer la forme de notre fonction de coût que nous allons chercher à minimiser

thetaRange = np.arange(80-100,80+100,1)
costFctEvol = pd.DataFrame({'theta':thetaRange,
                            'cost':[costFunction(data.y,hypothesis(data.x1,theta)) 
                                    for theta in thetaRange]})

fit, ax = plt.subplots(figsize=(16,10))
plt.plot(costFctEvol.theta,costFctEvol.cost)
plt.xlabel('theta',fontSize=15)
plt.ylabel('J(theta)',fontSize=15)
plt.show();

# 5) La descente de Gradient

* ## On utilise la dérivée de la fonction de coût: $\frac{d}{d\theta_{1}}J(\theta)$

    * ### Si $J(\theta)$ est croissant: $\frac{d}{d\theta_{1}}J(\theta) > 0$
    * ### Si $J(\theta)$ est décroissant: $\frac{d}{d\theta_{1}}J(\theta) < 0$

In [ ]:
# La descente de gradient utilise la notion de dérivée, 
#      illustrée ici avec la fonction carré (qui doit nous en rappeler une autre!)

def fct(x):
    return np.power(x,2)

def fctDeriv(x):
    return 2*x

fctCarre = pd.DataFrame({'x':np.arange(-10,10,0.1),'y':[fct(x) for x in np.arange(-10,10,0.1)]})
fctCarreD = pd.DataFrame({'x':np.arange(-10,10,0.1),
                          'y':[fctDeriv(x) for x in np.arange(-10,10,0.1)]})
fit, ax = plt.subplots(figsize=(16,10))
plt.plot(fctCarre.x,fctCarre.y,label='f(x)')
plt.plot(fctCarreD.x,fctCarreD.y,label="f'(x)")
plt.grid(True)
plt.legend(fontsize=15);

## Dérivée de la somme quadratique des erreurs:


* ### $J(\theta) = \frac{1}{2m} \displaystyle\sum_{i=0}^{m}(\hat{y}^{(i)} - y^{(i)})^{2} = \frac{1}{2m} \displaystyle\sum_{i=0}^{m}(\theta_{1}x_{1}^{(i)} - y^{(i)})^{2}$
* ### $\frac{d}{d\theta_{1}}J(\theta) = \frac{1}{m}\displaystyle\sum_{i=0}^{m}(\hat{y}^{(i)} - y^{(i)}) x_{1}^{(i)}$

In [ ]:
# La descente de gradient utilise la dérivée de la fonction de coût 
#    par rapport au paramètre theta1

def costFctDeriv(x,y,yhat):
    return ((yhat - y)*x.T).sum(axis=1)/y.shape[0]

## Algorithme de la descente de gradient:

### $\begin{matrix} \text{Répéter jusqu'à convergence:} & \{ & \\ & & \theta_{1} := \theta_{1} - \alpha \frac{d}{d\theta_{1}}J(\theta) \\ & \} & \end{matrix}$

In [ ]:
# À chaque étape de la descente de gradient (jusqu'à la convergence), 
#   on incremente la valeur de theta1 par ce résultat.
#   Alpha est le learning rate

def gradDescent(x,y,yhat,alpha):
    return -alpha*costFctDeriv(x,y,yhat)

## Préparation du dataset:

* ### On sépare aléatoirement les données en 2 (3) échantillons:
    * Entraînement / (Validation) / Test
    * 80 / 20 (70 / 30) ou 60 / 20 / 20

* ### Entraînement: utilisé pour la descente de gradient
* ### Validation: utilisé pour l'hyperparamètrage de l'algo
* ### Test: utilisé pour mesurer la performance du modèle

In [ ]:
# Split train/test
index = data.index.values.copy()
random.shuffle(index)

X_train = data.x1.loc[index[:int(len(index)*0.7)]]
X_test = data.x1.loc[index[int(len(index)*0.7):]]
Y_train = data.y.loc[index[:int(len(index)*0.7)]]
Y_test = data.y.loc[index[int(len(index)*0.7):]]

X_train.index = np.arange(X_train.shape[0])
Y_train.index = np.arange(Y_train.shape[0])
X_test.index = np.arange(X_test.shape[0])
Y_test.index = np.arange(Y_test.shape[0])

print("Train set X shape: {}".format(X_train.shape))
print("Train set Y shape: {}".format(Y_train.shape))
print("Test set X shape: {}".format(X_test.shape))
print("Test set X shape: {}".format(Y_test.shape))

## Hand-tunning (hyperparamètrage)

* ### Learning rate: $\alpha$ = 0.045 (pour la démo), plus efficace avec $\alpha$ = 0.03
* ### Précision (stop l'apprentissage): $\epsilon$ = 0.01

In [ ]:
# On utilise donc une valeur de départ pour theta généré aléatoirement entre 0 et 1, 
#    la valeur du learning rate est fixé à 0.00000003
# Epsilon correspond à la précision que l'on veut atteindre pour stopper la descente de gradient

thetaInit = np.random.rand()
yhat = hypothesis(X_train,thetaInit)
alpha = 0.045
epsilon = 0.01

In [ ]:
# On prepare un dataframe pour stocker les valeurs de J(theta) et theta1

gradDescentEvol = pd.DataFrame({'theta':thetaInit,
                                'J':costFunction(Y_train,yhat)},index = np.arange(1))

In [ ]:
# Convert X_train into dataframe
X_train = pd.DataFrame({"x1": X_train})

In [ ]:
# On parametrise deux trois trucs

plt.rcParams['figure.figsize'] = [16, 5]
costFct = 0
count = 0
theta = thetaInit

# Et on se lance dans la boucle: La descente de gradient!
while np.abs(costFunction(Y_train,yhat) - costFct) >= epsilon*costFct:
    count += 1
    costFct = costFunction(Y_train,yhat)
    theta += gradDescent(X_train,Y_train,yhat,alpha)
    yhat = hypothesis(X_train,theta)
    gradDescentEvol = gradDescentEvol.append(pd.DataFrame({'theta':theta['x1'],
                                                           'J':costFunction(Y_train,yhat)},
                                                          index = np.arange(1)),
                                             ignore_index=True)
    fig, ax = plt.subplots(ncols=3)
    plotData(ax[0],X_train['x1'],Y_train,theta['x1'],yhat,gradDescentEvol)
    plotCostFunction(ax[1],X_train['x1'],Y_train,theta['x1'],gradDescentEvol,thetaInit)
    plotCostFunctionEvol(ax[2],gradDescentEvol)
    clear_output(wait=True)
    display(plt.gcf())
    plt.close(fig)

In [ ]:
theta

# 6) Prédictions

In [ ]:
def errorFct(y,yhat):
    return np.abs(yhat - y).sum()/(y.shape[0])

In [ ]:
# Afficher les résultats:
print('La descente de gradient a été réalisé en %i étapes.' % count)
print('theta = %f' % theta)
print("\nTrain set:")
print('J(theta) = %f' % costFunction(Y_train,hypothesis(X_train['x1'],theta['x1'])))
print('Error = %f' % errorFct(Y_train, hypothesis(X_train,theta)))
print("\nTest set:")
print('J(theta) = %f' % costFunction(Y_test,hypothesis(X_test,theta['x1'])))
print('Error = %f' % errorFct(Y_test, hypothesis(X_test,theta['x1'])))

In [ ]:
# Faisons une prédiction ....

newGPUs = [5,10,14]
for newGPU in newGPUs:
    print("Notre nouvelle carte de %i Go de GPU pourra se vendre autour de %.2f €" % 
          (newGPU,newGPU*theta))
    
plt.rcParams['figure.figsize'] = [14, 8]
plt.plot(X_train,Y_train,'o',label='data')
plt.plot(getHypothesisForPLot(theta).x,getHypothesisForPLot(theta).y,'r',label='hypothèse')
for i in range(X_train.shape[0]):
    plt.plot((X_train['x1'][i],X_train['x1'][i]), (min(Y_train[i],yhat[i]),max(Y_train[i],yhat[i])), 'k-')
plt.plot(newGPUs,[newGPU*theta for newGPU in newGPUs], 'or', label='prédictions')
plt.xlabel('GPU (Go)',fontsize=15)
plt.ylabel('prix (€)',fontsize=15)
plt.legend(fontsize=15)
plt.show();

# 7) Régression linéaire multivariable

In [ ]:
# Get data (again...)
data = pd.read_csv('data/graphicCards.csv')
data.head()

## Data exploration: Deal with missing data

In [ ]:
data.isnull().sum()

In [ ]:
# Missing variables (NaN)
print("There are empty columns? {}".format(data.isnull().all().any()))
print("There are columns with mising values? {}".format(data.isnull().any().any()))
print("\nWhich ones?:\n")
print(data[data.columns[data.isnull().any()].values].isnull().sum())

In [ ]:
# Take look at the chipset feature
print("Unique values: {}\n".format(data.chipset.unique()))
print("Count values: \n")
print(data.chipset.value_counts())

In [ ]:
# What is the nan chipset entry?
data[data.chipset.isnull()]

In [ ]:
# What can we do? We drop it here (not interesting for our purpose). Don't forget to update indexing
print("Data shape before droping: {}".format(data.shape))
data = data[data.chipset.notnull()]
data.index = np.arange(data.shape[0])
print("Data shape after droping: {}".format(data.shape))
print("\nWhat are columns with missing values now?:\n")
print(data[data.columns[data.isnull().any()].values].isnull().sum())

In [ ]:
# BoostFreq features is NaN when the constructor don't provide information.
# One can seen that as the given graphic card has not boosting option
# Solution is to set this value to frequency value if NaN (then we'll deal with NaN frequency entries)
data.loc[data['boostFreq (MHz)'].isnull(),'boostFreq (MHz)'] = data.loc[data['boostFreq (MHz)'].isnull(),
                                                                        'frequency (MHz)']
print("\nWhat are columns with missing values now?:\n")
print(data[data.columns[data.isnull().any()].values].isnull().sum())

In [ ]:
# What about the multi GPU?
print("Unique values: {}\n".format(data['multi GPU'].unique()))
print("Count values: \n")
print(data['multi GPU'].value_counts())

In [ ]:
# SLI is the nvidia multi GPU technology while CrossFireX is the AMD one
# As distinction beetween AMD and Nvidia is given with chipset features, 
#    one could transform multi GPU feature to a boolean one:
#       - True if card has multi GPU technology (SLI or CrossFireX)
#       - False if not (NaN values)
data['multi GPU'] = data['multi GPU'].notnull()
print("multi GPU feature exemples: {}\n".format(data['multi GPU'].values[:10]))
print("What are columns with missing values now?:\n")
print(data[data.columns[data.isnull().any()].values].isnull().sum())

In [ ]:
# So, what about the last NaN values?
# Technicaly, frequency and boostFreq NaN corresponds to the same entries
# Print all NaN entries
data[data.isnull().any(axis=1)]

In [ ]:
# There are several way to deal with that:
#       -> find information from other source (constructor site)
#       -> estimate missing values from data (could introduce biais!)
#       -> drop them (only if there represent a small part of data)
#
# Here we'll drop them for simplicity
print("We'll drop {} entries ({}%)".format(data[data.isnull().any(axis=1)].shape[0],
                                           (data[data.isnull().any(axis=1)].shape[0]/data.shape[0])*100))
print("\nData shape before droping: {}".format(data.shape))
data = data[data.notnull().all(axis=1)]
data.index = np.arange(data.shape[0])
print("Data shape after droping: {}\n".format(data.shape))
print("There are empty columns? {}".format(data.isnull().all().any()))
print("There are columns with mising values? {}".format(data.isnull().any().any()))

## Data exploration: Features engineering

In [ ]:
# Let print first rows of our data
data.head()

In [ ]:
# How to transform our categorial features?
# Several things are possible, let's try to transform them as boolean features
#  -> First get number of uniques values for each categorials features
categorialFeatures = ['builder','chipset','overclok','bus','memory type','direct X (max)']
for feature in categorialFeatures:
    print("\n{}: ({} unique values)".format(feature, len(data[feature].unique())))
    print(data[feature].unique())

In [ ]:
# Here, we spot an error in overclok columns: 'no ' with a space. 
# It seems to be a typo, let replace it by correct 'no' value
data.loc[data.overclok == 'no ','overclok'] = 'no'
 
# Other problem: directX == 11.2 -> set it to 11
data.loc[data['direct X (max)'] == 11.2,'direct X (max)'] = 11

for feature in categorialFeatures:
    print("\n{}: ({} unique values)".format(feature, len(data[feature].unique())))
    print(data[feature].unique())

In [ ]:
# Set direct X as string type
data['direct X (max)'] = data['direct X (max)'].astype(str)

In [ ]:
# Features with only 2 possible values -> Set booleans
data['chipset_amd'] = (data['chipset'] == 'amd') * 1
data['overclock'] = (data['overclok'] == 'yes') * 1
data['direct_X_11'] = (data['direct X (max)'] == '11.0') * 1

data.drop(['chipset', 'overclok', 'direct X (max)'], axis=1, inplace=True)

categorialFeatures.remove('chipset')
categorialFeatures.remove('overclok')
categorialFeatures.remove('direct X (max)')

In [ ]:
# Let's create our boolean features (using get_dummies)
data = pd.concat([data.drop(categorialFeatures, axis=1), pd.get_dummies(data[categorialFeatures])], axis=1)    
print("New data shape: {}".format(data.shape))

In [ ]:
# Now replace boolean values (True, False) by numeric number (1, 0)
data[data.columns[data.dtypes == bool]] = data[data.columns[data.dtypes == bool]] * 1

data.head()

In [ ]:
# Last feature engineering transformation: replace boostFreq feature by the diff beetween frequency and boostFreq
data['boostFreqIncr'] = data['boostFreq (MHz)'] - data['frequency (MHz)']
data.drop('boostFreq (MHz)',axis=1,inplace=True)

In [ ]:
# Last things, make sure index are correct
data.index = np.arange(data.shape[0])
# That's ok now! Let print new data shape and first rows
print("Data shape = {}".format(data.shape))
data.head()

## Normalize dataset

In [ ]:
# Let store our Y vector (price) in separate Series
Y = data['price (euros)']
data.drop('price (euros)',axis=1,inplace=True)

In [ ]:
# Before normalize inputs, let's print a description of non-boolean features
data.describe()

In [ ]:
# As boostFreqIncr features has negatives values, in MinMax scaler is not a good thing to do.
# Instead we'll use a standard scaler (remove mean, divide by standard deviation)
data[data.columns[data.dtypes != object]] = (data[data.columns[data.dtypes != object]] - \
                                             data[data.columns[data.dtypes != object]].mean()) / \
                                            data[data.columns[data.dtypes != object]].std()
data.head()

In [ ]:
# Add bias: coef directeur -> Une GPU n'est jamais gratuite !
data['x0'] = 1
data.head()

## Train / test splitting

In [ ]:
# We need to separate our dataset into two sub-sample:
#    -> train set: used to train the model (70%)
#    -> test set: to measure performances (30%)
# Let pick random entries
index = data.index.values.copy()
random.shuffle(index)

X_train = data.loc[index[:int(len(index)*0.7)]]
X_test = data.loc[index[int(len(index)*0.7):]]
Y_train = Y.loc[index[:int(len(index)*0.7)]]
Y_test = Y.loc[index[int(len(index)*0.7):]]

print("Train set X shape: {}".format(X_train.shape))
print("Train set Y shape: {}".format(Y_train.shape))
print("Test set X shape: {}".format(X_test.shape))
print("Test set X shape: {}".format(Y_test.shape))

## Multivariable linear regression

In [ ]:
# On génère aléatoirement le vecteur des paramétres de notre modèle
theta = np.random.rand(X_train.shape[1])-0.5

# Compute cost function value for this initialized model
yhat = hypothesis(X_train,theta)
print("Cost Function value = {}".format(costFunction(Y_train,yhat)))

In [ ]:
# Now it's time to run the gradient descent, let's parametrize it
alpha = 0.03
epsilon = 0.01
gradDescentEvol = pd.DataFrame()
count = 0
costFct = 0
plot = True

# and run it! (it's a quite long...)
while np.abs(costFunction(Y_train,yhat) - costFct) >= epsilon * costFct:
    count += 1
    costFct = costFunction(Y_train,yhat)
    theta += gradDescent(X_train,Y_train,yhat,alpha)
    yhat = hypothesis(X_train,theta)
    gradDescentEvol = gradDescentEvol.append(pd.DataFrame({'Jtrain':costFunction(Y_train,yhat),
                                                           'Jtest':costFunction(Y_test,hypothesis(X_test,theta))},
                                                          index = np.arange(1)),
                                             ignore_index=True)
    if plot:
        fig, ax = plt.subplots(figsize=(10,5))
        plotCostFunctionEvolwTest(ax,gradDescentEvol)
        clear_output(wait=True)
        display(plt.gcf())
        plt.close(fig)

In [ ]:
# Afficher les résultats:
print('La descente de gradient a été réalisé en %i étapes.' % count)
print('train_err = %f' % errorFct(Y_train,hypothesis(X_train,theta)))
print('test_err = %f' % errorFct(Y_test,hypothesis(X_test,theta)))

In [ ]:
theta.sort_values()

In [ ]:
theta[[c for c in theta.keys() if 'builder' in c]].sort_values()